In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

def get_price_info(dfcards):

    price_list = []
    change_list = []
    sales_list = []

    for game_id in dfcards["game_id"]:

        print(f"Fetching data for game ID: {game_id}")

        change = None
        sales_per_day = None

        url = f"https://www.pricecharting.com/game/{game_id}"
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                        "AppleWebKit/537.36 (KHTML, like Gecko) "
                        "Chrome/122.0.0.0 Safari/537.36"
        }
        response = requests.get(url, headers=headers)

        if response.status_code != 200:
            print(f"Failed to fetch {url}")
        
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Get price
        price_tag = soup.find("span", class_="price js-price")
        price = price_tag.text.strip()[1:] if price_tag else None

        # Get change
        change_container = soup.find("span", class_="change", title=True)
        if change_container:
            sign = '+' if '+' in change_container.text else '-' if '-' in change_container.text else ''
            change_tag = change_container.find("span", class_="js-price")
            change = sign + change_tag.text.strip()[1:] if change_tag else None
        else:
            change = None

        # Get sales per day
        sales_tag = None
        td_tags = soup.find_all("td", class_="js-show-tab", attrs={"data-show-tab": "completed-auctions-used"})
        for td in td_tags:
            link = td.find("a")
            if link and "sales per day" in link.text:
                sales_tag = link
                break
        sales_per_day = sales_tag.text.strip().split()[0] if sales_tag else 0

        # Append the data to the lists
        price_list.append(float(price) if price else None)
        change_list.append(float(change) if change else None)
        sales_list.append(float(sales_per_day))
    
    # Update dataframe with the new columns
    dfcards["price"] = price_list
    dfcards["change"] = change_list
    dfcards["sales_per_day"] = sales_list
    return dfcards 

In [ ]:
def dfcards_from_csv(csv_file, skip_nan=True):

    # Read the CSV file
    df = pd.read_csv(csv_file, sep=';', header=0, encoding='utf-8')
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    
    if skip_nan:
        # Remove rows where either column contains NaN
        df = df.dropna(subset=df.columns[:2])

    return df

def get_game_ids(dfcards_orig):

    dfcards = dfcards_orig.copy()

    card_names = dfcards.Card.tolist()
    card_sets = dfcards.Set.tolist()

    game_ids = []
    for name, set_name in zip(card_names, card_sets):
        # Format the set name (lowercase, replace spaces with hyphens)
        formatted_set = f"pokemon-{set_name.lower().replace(' ', '-')}"
        
        # Extract number if it exists
        if '#' in name:
            name, number = name.split('#')
            # Format card name (lowercase, trim spaces, replace spaces with hyphens)
            formatted_name = name.lower().strip().replace(' ', '-')
            # Remove any spaces from number
            formatted_number = number.strip()
            # Combine into pricecharting format
            game_id = f"{formatted_set}/{formatted_name}-{formatted_number}"
        else:
            # Handle cases without number
            formatted_name = name.lower().strip().replace(' ', '-')
            game_id = f"{formatted_set}/{formatted_name}"
            
        game_ids.append(game_id)

    dfcards['game_id'] = game_ids
    
    return dfcards

In [ ]:
df_csv = dfcards_from_csv('PokemonTCG.csv', skip_nan=False)

In [ ]:
df_fetch = dfcards_from_csv('PokemonTCG.csv', skip_nan=True)

In [ ]:
df_fetch 

In [ ]:
df = get_price_info(get_game_ids(dfcards_from_csv('PokemonTCG.csv')))

In [ ]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

df_final = df_csv.merge(df[['price','change','sales_per_day']], left_index=True, right_index=True, how='left')

In [ ]:
df_final.to_csv('PokemonTCG_final.csv', sep=';', index=False, encoding='utf-8')

In [ ]:
print(df['price'].to_string(index=False))